# ViT 底层代码逻辑理解

我们都知道 Vision Transformer 让 Transformer 在 NLP 中注意力的思想进入了 CV 之中。不过，ViT 在 Google 开源的时候，不是 pytorch 版本。这让 pytorch 熟练却 jax / tensorflow 生疏者小不适应。

[官方Google代码传送门](https://github.com/google-research/vision_transformer)

幸运的是找到一个数十万 star 的 pytorch 复现版本！[传送门](https://github.com/lucidrains/vit-pytorch)

我们来看一看吧

In [26]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from einops import rearrange, repeat
from einops.layers.torch import Rearrange

# helpers

def pair(t):
    return t if isinstance(t, tuple) else (t, t)

class FeedForward(nn.Module):
    def __init__(self, dim, hidden_dim, dropout = 0.1):
        super().__init__()
        self.net = nn.Sequential(
            nn.LayerNorm(dim),
            nn.Linear(dim, hidden_dim),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, dim),
            nn.Dropout(dropout)
        )

    def forward(self, x):
        return self.net(x)

这些都没的说， `FeedForward` 这个class 也就是两层 MLP，中间一个 LN，激活函数 GELU 和 Dropout。下面是一个 MHA。

In [27]:
class MultiHeadAttention(nn.Module):
    
    def __init__(self, dim, heads = 8, dim_head = 64, dropout = 0., need_mask=False) -> None:
        super(MultiHeadAttention, self).__init__()
        #! 当 num_heads == 1 时，就相当于单头注意力。如注意力是单头，则没有最后的 project MLP。
        #! dim 是 数据空间，不是内部 embed 的空间。
        inner_dim = dim_head * heads
        project_out = not (heads == 1 and dim_head == dim)
        
        self.heads = heads
        self.scale = dim_head ** -0.5 #& 经典的 Transformer 必问之根号 d，注意 这个 d 是 内部 embed 的 dim

        self.norm = nn.LayerNorm(dim)
        # self.softmax = nn.Softmax(dim = -1)
        self.dropout = nn.Dropout(dropout)

        self.to_qkv = nn.Linear(dim, inner_dim * 3, bias = False)
        self.to_out = nn.Sequential(
            nn.Linear(inner_dim, dim),
            nn.Dropout(dropout)
        ) if project_out else nn.Identity()

        self.need_mask = need_mask

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.norm(x)  #& Pre-Norm，更适合深层模型
        assert x.ndim == 3, "x 必须是 [B, L, D] 的形式！"
        B, L, _ = x.shape
        q, k, v = self.to_qkv(x).chunk(3, dim = -1)

        # q, k, v = map(lambda t: rearrange(t, 'b l (h d) -> b h l d', h = self.num_heads), (q, k, v))
        #* [B, L, D] -> [B, L, h, d] -> [B, h, L, d]  
        #* 注意这里直接 [B, L, D] -> [B, h, L, d] 也就是 q.reshape(B, self.num_heads, L, inner_dim // self.num_heads) 是不对的！
        
        q = q.reshape(B, L, self.heads, -1).permute(0, 2, 1, 3) 
        k = k.reshape(B, L, self.heads, -1).permute(0, 2, 1, 3)
        v = v.reshape(B, L, self.heads, -1).permute(0, 2, 1, 3) 

        dots = torch.matmul(q, k.transpose(-1, -2)) * self.scale
        #& Causal mask，请注意在计算 Softmax 概率分布前，算完点积相似度得分之后
        if self.need_mask:
            mask = torch.triu(torch.ones(L, L, device=x.device), diagonal=1).bool()  # 上三角为True
            dots = dots.masked_fill(mask[None, None, :, :], -torch.inf)  # 广播到所有batch和head

        
        dots = F.softmax(dots, dim = -1)
        dots = self.dropout(dots)
        out = torch.matmul(dots, v)

        out = out.transpose(1, 2).reshape(B, L, -1) #* 再回到 [B, h, L, d] -> [B, L, h*d]
        return self.to_out(out)



这个其实就是最经典的手撕 MHA 了，肯定也是被无数大厂考过 N 遍了。

刚刚已经入门 pytorch 的人肯定都知道，要让 `shape` 对上，才可以让代码run 通，但是实际上只顾torch的shape对不对的上是不够的。下面我们演示一个例子。

我们假设有一个 `B=2, L=5, D=6` 的Tensor，就是2个样本，每个样本序列长度5，在每个长度维度上是 6维向量。

按照 MHA的机制，假设有 2 个head，把 D 拆分成 2 个 3，那应该是对所有的 1-3 维度的 dim，和 4-6维 的dim，这两个去做一个Attention。

In [28]:
# 下面给出一个例子，假设 a.bc，这里 a 位置表示的是 batch size，而 b 位置表示 sequence length，c 位置表示的是 dimension。
tensor = torch.tensor([
    [
        [1.11, 1.12, 1.13, 1.14, 1.15, 1.16],  # L=1
        [1.21, 1.22, 1.23, 1.24, 1.25, 1.26],  # L=2
        [1.31, 1.32, 1.33, 1.34, 1.35, 1.36],  # L=3
        [1.41, 1.42, 1.43, 1.44, 1.45, 1.46],  # L=4
        [1.51, 1.52, 1.53, 1.54, 1.55, 1.56]   # L=5
    ],
    [
        [2.11, 2.12, 2.13, 2.14, 2.15, 2.16],  # L=1
        [2.21, 2.22, 2.23, 2.24, 2.25, 2.26],  # L=2
        [2.31, 2.32, 2.33, 2.34, 2.35, 2.36],  # L=3
        [2.41, 2.42, 2.43, 2.44, 2.45, 2.46],  # L=4
        [2.51, 2.52, 2.53, 2.54, 2.55, 2.56]   # L=5
    ]
])
#* 正确写法：[B, L, D] -> [B, L, h, d] -> [B, h, L, d]
tensor.reshape(2, 5, 2, 3).permute(0, 2, 1, 3)

tensor([[[[1.1100, 1.1200, 1.1300],
          [1.2100, 1.2200, 1.2300],
          [1.3100, 1.3200, 1.3300],
          [1.4100, 1.4200, 1.4300],
          [1.5100, 1.5200, 1.5300]],

         [[1.1400, 1.1500, 1.1600],
          [1.2400, 1.2500, 1.2600],
          [1.3400, 1.3500, 1.3600],
          [1.4400, 1.4500, 1.4600],
          [1.5400, 1.5500, 1.5600]]],


        [[[2.1100, 2.1200, 2.1300],
          [2.2100, 2.2200, 2.2300],
          [2.3100, 2.3200, 2.3300],
          [2.4100, 2.4200, 2.4300],
          [2.5100, 2.5200, 2.5300]],

         [[2.1400, 2.1500, 2.1600],
          [2.2400, 2.2500, 2.2600],
          [2.3400, 2.3500, 2.3600],
          [2.4400, 2.4500, 2.4600],
          [2.5400, 2.5500, 2.5600]]]])

In [29]:
#! 错误解法：直接 [B, L, D] -> [B, h, L, d]
tensor.reshape(2, 2, 5, 3)

tensor([[[[1.1100, 1.1200, 1.1300],
          [1.1400, 1.1500, 1.1600],
          [1.2100, 1.2200, 1.2300],
          [1.2400, 1.2500, 1.2600],
          [1.3100, 1.3200, 1.3300]],

         [[1.3400, 1.3500, 1.3600],
          [1.4100, 1.4200, 1.4300],
          [1.4400, 1.4500, 1.4600],
          [1.5100, 1.5200, 1.5300],
          [1.5400, 1.5500, 1.5600]]],


        [[[2.1100, 2.1200, 2.1300],
          [2.1400, 2.1500, 2.1600],
          [2.2100, 2.2200, 2.2300],
          [2.2400, 2.2500, 2.2600],
          [2.3100, 2.3200, 2.3300]],

         [[2.3400, 2.3500, 2.3600],
          [2.4100, 2.4200, 2.4300],
          [2.4400, 2.4500, 2.4600],
          [2.5100, 2.5200, 2.5300],
          [2.5400, 2.5500, 2.5600]]]])

实际上使用 `einops.rearrange` 函数可以很方便地将一个张量的形状进行变换。结果正确

In [30]:
rearrange(tensor, 'b l (h d) -> b h l d', h=2)

tensor([[[[1.1100, 1.1200, 1.1300],
          [1.2100, 1.2200, 1.2300],
          [1.3100, 1.3200, 1.3300],
          [1.4100, 1.4200, 1.4300],
          [1.5100, 1.5200, 1.5300]],

         [[1.1400, 1.1500, 1.1600],
          [1.2400, 1.2500, 1.2600],
          [1.3400, 1.3500, 1.3600],
          [1.4400, 1.4500, 1.4600],
          [1.5400, 1.5500, 1.5600]]],


        [[[2.1100, 2.1200, 2.1300],
          [2.2100, 2.2200, 2.2300],
          [2.3100, 2.3200, 2.3300],
          [2.4100, 2.4200, 2.4300],
          [2.5100, 2.5200, 2.5300]],

         [[2.1400, 2.1500, 2.1600],
          [2.2400, 2.2500, 2.2600],
          [2.3400, 2.3500, 2.3600],
          [2.4400, 2.4500, 2.4600],
          [2.5400, 2.5500, 2.5600]]]])

从这里看出，两种写法虽然shape对，但结果是很大的不同，直接`reshape`虽然不报错，但是机理上是大错特错。在实现 MHA 的时候一定要注意。

In [31]:
class Transformer(nn.Module):
    #! 这个 Transformer 就是 Decoder_only 架构。
    def __init__(self, dim, depth, heads, dim_head, mlp_dim, dropout = 0.):
        super().__init__()
        self.norm = nn.LayerNorm(dim)
        self.layers = nn.ModuleList([])
        for _ in range(depth):
            self.layers.append(nn.ModuleList([
                MultiHeadAttention(dim, heads = heads, dim_head = dim_head, dropout = dropout),
                FeedForward(dim, mlp_dim, dropout = dropout)
            ]))

    def forward(self, x):
        for attn, ff in self.layers:
            x = attn(x) + x
            x = ff(x) + x

        return self.norm(x)

**上面半段都是 Transformer 的实现，下面正片开始进入 ViT！**

In [61]:
class ViT(nn.Module):
    def __init__(self, *, image_size, patch_size, num_classes, dim, depth, num_heads, mlp_dim, pool = 'cls', channels = 3, dim_head = 64, dropout = 0., emb_dropout = 0.):
        super().__init__()
        image_height, image_width = pair(image_size)
        patch_height, patch_width = pair(patch_size)
        #! 假设图像的宽高一样，patch 大小的宽高也一样。

        assert image_height % patch_height == 0 and image_width % patch_width == 0, 'Image dimensions must be divisible by the patch size.'

        num_patches = (image_height // patch_height) * (image_width // patch_width)
        patch_dim = channels * patch_height * patch_width
        assert pool in {'cls', 'mean'}, 'pool type must be either cls (cls token) or mean (mean pooling)'

        self.to_patch_embedding = nn.Sequential(
            Rearrange('b c (h p1) (w p2) -> b (h w) (p1 p2 c)', p1 = patch_height, p2 = patch_width),
            nn.LayerNorm(patch_dim),
            nn.Linear(patch_dim, dim),
            nn.LayerNorm(dim),
        )
        

        self.pos_embedding = nn.Parameter(torch.randn(1, num_patches + 1, dim))
        self.cls_token = nn.Parameter(torch.randn(1, 1, dim))
        self.dropout = nn.Dropout(emb_dropout)

        self.transformer = Transformer(dim, depth, num_heads, dim_head, mlp_dim, dropout)

        self.pool = pool
        self.to_latent = nn.Identity()

        self.mlp_head = nn.Linear(dim, num_classes)

    def forward(self, img):
        x = self.to_patch_embedding(img)
        b, n, _ = x.shape

        cls_tokens = repeat(self.cls_token, '1 1 d -> b 1 d', b = b)
        x = torch.cat((cls_tokens, x), dim=1)
        x += self.pos_embedding[:, :(n + 1)]
        x = self.dropout(x)

        x = self.transformer(x)
        x = x.mean(dim = 1) if self.pool == 'mean' else x[:, 0] #* 或者选择 class token 第一个，或者是取平均。
        x = self.to_latent(x)
        return self.mlp_head(x)
    
v = ViT(
    image_size = 256,
    patch_size = 32, #! 每小块 32 的大小，总图像 256，从而划分为 8 * 8 = 64 个小块。
    num_classes = 1000,
    dim = 1024,
    depth = 6,
    num_heads = 16,
    mlp_dim = 2048,
    dropout = 0.1,
    emb_dropout = 0.1
)

img = torch.randn(4, 3, 256, 256)
v(img).shape

x after transformer: torch.Size([4, 65, 1024])
x: torch.Size([4, 1024])


torch.Size([4, 1000])

run 通成功！解析一下

1. `patch_dim` 是把 `channel` 也乘进去了。就是 `channel * patch_hehight * patch_width`。
2. `[B, C, H, W] -> [B, C, h, p1, W, p2] -> [B, h, w, p1, p2, C] -> [B, h * w, p1 * p2, C]`
3. 第一步，先把图像 patchify 后 embed 进入，`[B, C, H, W] -> [B, h*w, D]` 这里 `B, L, D` 中 `L = h*w`。相当于图像的每一个 patch 为序列长度维度。

下面我们说一说 `class_token`：其是一个 `randn` 初始化可训练的参数。初始shape 为 `[1, 1, D]`，但对于一个batch而言这个重复 `B` 次。
- 从而然后和 `[B, h*w, D]` 拼接起来 为 `[B, h*w+1, D]`。
- 位置编码参数，`randn` 初始化，shape 为 `[1, h*w+1, D]` 就是说 `x += pos`
- 随后进入 Transformer。
- 离开 Transformer 之后，仍然得到 `[B, h*w+1, D]`。这个时候可能是 对 `h*w + 1` 这一维度求平均，变为 `[B, D]`。
- 最后经过 `nn.Linear`，得到 `[B, num_classes]`。

这也就是 ViT 分类图像的底层代码逻辑了。

下面我们来解释里面 ViT 一些变换操作。主要核心肯定在这一块需要扒一下：

假设我有一个 `(2, 3, 6, 6)` 的 rgb tensor，size 为 4，然后 patchify 为 2。

In [39]:
input_tensor = torch.zeros(2, 3, 6, 6)

# 填充数值规律：
# - Batch 1: 所有值以 1.xxx 开头，按 h 和 w 递增
# - Batch 2: 所有值以 2.xxx 开头（仅示例，实际可按需修改）
for b in range(2):
    for c in range(3):
        for h in range(6):
            for w in range(6):
                # 数值生成规则：b+1 . h+1 w+1 c+1
                input_tensor[b, c, h, w] = float(f"{b+1}.{c+1}{h+1}{w+1}")
input_tensor

tensor([[[[1.1110, 1.1120, 1.1130, 1.1140, 1.1150, 1.1160],
          [1.1210, 1.1220, 1.1230, 1.1240, 1.1250, 1.1260],
          [1.1310, 1.1320, 1.1330, 1.1340, 1.1350, 1.1360],
          [1.1410, 1.1420, 1.1430, 1.1440, 1.1450, 1.1460],
          [1.1510, 1.1520, 1.1530, 1.1540, 1.1550, 1.1560],
          [1.1610, 1.1620, 1.1630, 1.1640, 1.1650, 1.1660]],

         [[1.2110, 1.2120, 1.2130, 1.2140, 1.2150, 1.2160],
          [1.2210, 1.2220, 1.2230, 1.2240, 1.2250, 1.2260],
          [1.2310, 1.2320, 1.2330, 1.2340, 1.2350, 1.2360],
          [1.2410, 1.2420, 1.2430, 1.2440, 1.2450, 1.2460],
          [1.2510, 1.2520, 1.2530, 1.2540, 1.2550, 1.2560],
          [1.2610, 1.2620, 1.2630, 1.2640, 1.2650, 1.2660]],

         [[1.3110, 1.3120, 1.3130, 1.3140, 1.3150, 1.3160],
          [1.3210, 1.3220, 1.3230, 1.3240, 1.3250, 1.3260],
          [1.3310, 1.3320, 1.3330, 1.3340, 1.3350, 1.3360],
          [1.3410, 1.3420, 1.3430, 1.3440, 1.3450, 1.3460],
          [1.3510, 1.3520, 1.3530, 1

In [40]:
rearrange(input_tensor, 'b c (h p1) (w p2) -> b (h w) (p1 p2 c)', p1=2, p2=2)

tensor([[[1.1110, 1.2110, 1.3110, 1.1120, 1.2120, 1.3120, 1.1210, 1.2210,
          1.3210, 1.1220, 1.2220, 1.3220],
         [1.1130, 1.2130, 1.3130, 1.1140, 1.2140, 1.3140, 1.1230, 1.2230,
          1.3230, 1.1240, 1.2240, 1.3240],
         [1.1150, 1.2150, 1.3150, 1.1160, 1.2160, 1.3160, 1.1250, 1.2250,
          1.3250, 1.1260, 1.2260, 1.3260],
         [1.1310, 1.2310, 1.3310, 1.1320, 1.2320, 1.3320, 1.1410, 1.2410,
          1.3410, 1.1420, 1.2420, 1.3420],
         [1.1330, 1.2330, 1.3330, 1.1340, 1.2340, 1.3340, 1.1430, 1.2430,
          1.3430, 1.1440, 1.2440, 1.3440],
         [1.1350, 1.2350, 1.3350, 1.1360, 1.2360, 1.3360, 1.1450, 1.2450,
          1.3450, 1.1460, 1.2460, 1.3460],
         [1.1510, 1.2510, 1.3510, 1.1520, 1.2520, 1.3520, 1.1610, 1.2610,
          1.3610, 1.1620, 1.2620, 1.3620],
         [1.1530, 1.2530, 1.3530, 1.1540, 1.2540, 1.3540, 1.1630, 1.2630,
          1.3630, 1.1640, 1.2640, 1.3640],
         [1.1550, 1.2550, 1.3550, 1.1560, 1.2560, 1.3560, 1.1650

实际上这个其实可以等价于一个操作：

In [57]:
input_tensor.reshape(2, 3, 3, 2, 3, 2).permute(0, 2, 4, 3, 5, 1).reshape(2, 9, 12)
#! [B, C, H, W] -> [B, C, h, p1, w, p2] -> [B, h, w, p1, p2, C] -> [B, h*w, p1*p2*C]
torch.allclose(input_tensor.reshape(2, 3, 3, 2, 3, 2).permute(0, 2, 4, 3, 5, 1).reshape(2, 9, 12), 
               rearrange(input_tensor, 'b c (h p1) (w p2) -> b (h w) (p1 p2 c)', p1=2, p2=2))

True

我们还同样不难推出 1D 2D 3D 的变化在于：

```py
# 1D
num_patches = seq_len // patch_size
patch_dim = channels * patch_size

self.to_patch_embedding = nn.Sequential(
    Rearrange('b c (n p) -> b n (p c)', p = patch_size),
    nn.LayerNorm(patch_dim),
    nn.Linear(patch_dim, dim),
    nn.LayerNorm(dim),
)


# 2D
num_patches = (image_height // patch_height) * (image_width // patch_width)
patch_dim = channels * patch_height * patch_width

self.to_patch_embedding = nn.Sequential(
    Rearrange('b c (h p1) (w p2) -> b (h w) (p1 p2 c)', p1 = patch_height, p2 = patch_width),
    nn.LayerNorm(patch_dim),
    nn.Linear(patch_dim, dim),
    nn.LayerNorm(dim),
)

# 3D
num_patches = (image_height // patch_height) * (image_width // patch_width) * (frames // frame_patch_size)
patch_dim = channels * patch_height * patch_width * frame_patch_size

self.to_patch_embedding = nn.Sequential(
    Rearrange('b c (f pf) (h p1) (w p2) -> b (f h w) (p1 p2 pf c)', p1 = patch_height, p2 = patch_width, pf = frame_patch_size),
    nn.LayerNorm(patch_dim),
    nn.Linear(patch_dim, dim),
    nn.LayerNorm(dim),
)
```

总的来说，第一步patchify 然后 embed 的过程里：
1. 把 每个维度的 patch 个数相乘在一个维度上，作为交给 Transformer 处理的 Sequence length 的维度
2. 把 每个维度的 patch 大小和 channel 放在最后一个维度上，作为 Transformer 处理的 Sequence 每个 element 的向量dim维度
3. 然后经过 MLP。前后各有一次 LN
